In [5]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
import ssl
from sqlalchemy import create_engine, inspect

ssl._create_default_https_context = ssl._create_unverified_context
# Import API key
from config import gkey

In [13]:
# Import College Football Stadiums file into the stadiums_pd DataFrame
url = 'https://en.wikipedia.org/wiki/List_of_NCAA_Division_I_FBS_football_stadiums'
stadiums_pd = pd.read_html(url)[0]

stadiums_pd = stadiums_pd.drop(columns="Image")

stadiums_pd = stadiums_pd.rename(columns={"Record1": "Record"})
stadiums_pd

,Stadium,City,State,Team,Conference,Capacity,Record,Built,Expanded 2,Surface
0,Aggie Memorial Stadium,Las Cruces,NM,New Mexico State,Independent,"30,343[1]","32,993(September 26, 1998 vs. UTEP)[1]",1978[1],2005[1],S5-M Synthetic Turf
1,Alamodome,San Antonio,TX,UTSA,C-USA,65000,"56,743(September 3, 2011 vs. Northeastern State)",1993,NaN,SportField
2,Alaska Airlines Field at Husky Stadium,Seattle,WA,Washington,Pac-12,"70,083[2]","76,125(September 23, 1995 vs. Army)",1920,2013,FieldTurf
3,Albertsons Stadium,Boise,ID,Boise State,Mountain West,"36,387[3]","36,864(September 20, 2012 vs. BYU)[4]",1970[3],2012[3],Smurf Turf
4,Allegiant Stadium,Paradise,NV,UNLV,Mountain West,65000,NaN,2020,NaN,FieldTurf
...,...,...,...,...,...,...,...,...,...,...
125,Veterans Memorial Stadium at Larry Blakeney Field,Troy,AL,Troy,Sun Belt,30402,"29,013(September 15, 2012 vs. Mississippi State)",1950,2003 & 2018,ProGrass
126,Waldo Stadium,Kalamazoo,MI,Western Michigan,MAC,"30,200[152]","36,361(September 16, 2000 vs. Indiana State)",1939[153],2003[153],FieldTurf
127,Warren McGuirk Alumni Stadium,Hadley,MA,UMass,Independent,"17,000(expandable to 21,430[154])","20,000(November 25, 1972 vs. Boston College)",1965,2014,FieldTurf
128,Wayne Day Family Field at Carter–Finley Stadium,Raleigh,NC,NC State,ACC,"57,583[155]","57,583(multiple times)",1966,2006,Tifway 419 Bermuda Grass


In [14]:
# Add columns for lat, lng
# Hint: Be sure to specify "" as the initial value for each column.

stadiums_pd['Lat'] = ""
stadiums_pd['Lng'] = ""

stadiums_ordered_pd = stadiums_pd[["Stadium","Lat","Lng","City","State","Team","Conference","Capacity","Record","Built","Surface"]]

stadiums_ordered_pd

,Stadium,Lat,Lng,City,State,Team,Conference,Capacity,Record,Built,Surface
0,Aggie Memorial Stadium,,,Las Cruces,NM,New Mexico State,Independent,"30,343[1]","32,993(September 26, 1998 vs. UTEP)[1]",1978[1],S5-M Synthetic Turf
1,Alamodome,,,San Antonio,TX,UTSA,C-USA,65000,"56,743(September 3, 2011 vs. Northeastern State)",1993,SportField
2,Alaska Airlines Field at Husky Stadium,,,Seattle,WA,Washington,Pac-12,"70,083[2]","76,125(September 23, 1995 vs. Army)",1920,FieldTurf
3,Albertsons Stadium,,,Boise,ID,Boise State,Mountain West,"36,387[3]","36,864(September 20, 2012 vs. BYU)[4]",1970[3],Smurf Turf
4,Allegiant Stadium,,,Paradise,NV,UNLV,Mountain West,65000,NaN,2020,FieldTurf
...,...,...,...,...,...,...,...,...,...,...,...
125,Veterans Memorial Stadium at Larry Blakeney Field,,,Troy,AL,Troy,Sun Belt,30402,"29,013(September 15, 2012 vs. Mississippi State)",1950,ProGrass
126,Waldo Stadium,,,Kalamazoo,MI,Western Michigan,MAC,"30,200[152]","36,361(September 16, 2000 vs. Indiana State)",1939[153],FieldTurf
127,Warren McGuirk Alumni Stadium,,,Hadley,MA,UMass,Independent,"17,000(expandable to 21,430[154])","20,000(November 25, 1972 vs. Boston College)",1965,FieldTurf
128,Wayne Day Family Field at Carter–Finley Stadium,,,Raleigh,NC,NC State,ACC,"57,583[155]","57,583(multiple times)",1966,Tifway 419 Bermuda Grass


In [15]:
params = {
    'key': gkey
}

    
for index, row in stadiums_ordered_pd.iterrows():
    
    base_url = f"https://maps.googleapis.com/maps/api/geocode/json"
    
    city = row['City']
    state = row['State']

    # add keyword to params dict
    params['address'] = f'{city},{state}'

    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        stadiums_ordered_pd.loc[index, 'Lat'] = results[0]['geometry']['location']['lat']
        stadiums_ordered_pd.loc[index, 'Lng'] = results[0]['geometry']['location']['lng']

        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

    
stadiums_ordered_pd.head()

C:\Users\chris\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\chris\Anaconda3\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------

,Stadium,Lat,Lng,City,State,Team,Conference,Capacity,Record,Built,Surface
0,Aggie Memorial Stadium,32.31994,-106.763654,Las Cruces,NM,New Mexico State,Independent,"30,343[1]","32,993(September 26, 1998 vs. UTEP)[1]",1978[1],S5-M Synthetic Turf
1,Alamodome,29.424122,-98.493628,San Antonio,TX,UTSA,C-USA,65000,"56,743(September 3, 2011 vs. Northeastern State)",1993,SportField
2,Alaska Airlines Field at Husky Stadium,47.606209,-122.332071,Seattle,WA,Washington,Pac-12,"70,083[2]","76,125(September 23, 1995 vs. Army)",1920,FieldTurf
3,Albertsons Stadium,43.615019,-116.202314,Boise,ID,Boise State,Mountain West,"36,387[3]","36,864(September 20, 2012 vs. BYU)[4]",1970[3],Smurf Turf
4,Allegiant Stadium,36.097195,-115.146665,Paradise,NV,UNLV,Mountain West,65000,NaN,2020,FieldTurf


In [17]:
from pprint import pprint

for index, row in stadiums_ordered_pd.iterrows():
    
    target_city = row['Stadium']
    
    base_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={target_city}&key={gkey}"
    
    lat = row['Lat']
    lng = row['Lng']

    # add keyword to params dict
    params['location'] = f'{lat},{lng}'
    
    # assemble 

    response = requests.get(base_url, params=params).json() 
    
    try:
        stadiums_ordered_pd.loc[index, 'Latitude'] = response['results'][0]['geometry']['location']['lat']
        stadiums_ordered_pd.loc[index, 'Longitude'] = response['results'][0]['geometry']['location']['lng'] 
    
    except:
        print(target_city)
    
stadiums_ordered_pd

,Stadium,Lat,Lng,City,State,Team,Conference,Capacity,Record,Built,Surface,Latitude,Longitude
0,Aggie Memorial Stadium,32.31994,-106.763654,Las Cruces,NM,New Mexico State,Independent,"30,343[1]","32,993(September 26, 1998 vs. UTEP)[1]",1978[1],S5-M Synthetic Turf,32.279486,-106.741713
1,Alamodome,29.424122,-98.493628,San Antonio,TX,UTSA,C-USA,65000,"56,743(September 3, 2011 vs. Northeastern State)",1993,SportField,29.416983,-98.478814
2,Alaska Airlines Field at Husky Stadium,47.606209,-122.332071,Seattle,WA,Washington,Pac-12,"70,083[2]","76,125(September 23, 1995 vs. Army)",1920,FieldTurf,47.650350,-122.301611
3,Albertsons Stadium,43.615019,-116.202314,Boise,ID,Boise State,Mountain West,"36,387[3]","36,864(September 20, 2012 vs. BYU)[4]",1970[3],Smurf Turf,43.602844,-116.195866
4,Allegiant Stadium,36.097195,-115.146665,Paradise,NV,UNLV,Mountain West,65000,NaN,2020,FieldTurf,36.090866,-115.183322
...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,Veterans Memorial Stadium at Larry Blakeney Field,31.808768,-85.969951,Troy,AL,Troy,Sun Belt,30402,"29,013(September 15, 2012 vs. Mississippi State)",1950,ProGrass,31.799556,-85.951833
126,Waldo Stadium,42.291707,-85.587229,Kalamazoo,MI,Western Michigan,MAC,"30,200[152]","36,361(September 16, 2000 vs. Indiana State)",1939[153],FieldTurf,42.285756,-85.601090
127,Warren McGuirk Alumni Stadium,42.341757,-72.588422,Hadley,MA,UMass,Independent,"17,000(expandable to 21,430[154])","20,000(November 25, 1972 vs. Boston College)",1965,FieldTurf,42.377271,-72.536040
128,Wayne Day Family Field at Carter–Finley Stadium,35.77959,-78.638179,Raleigh,NC,NC State,ACC,"57,583[155]","57,583(multiple times)",1966,Tifway 419 Bermuda Grass,35.800800,-78.719566


In [18]:
stadiums_cleaned_df = stadiums_ordered_pd[["Stadium", "Latitude", "Longitude", "City", "State", "Team", "Conference", "Capacity", "Record", "Built", "Surface"]]

stadiums_cleaned_df

,Stadium,Latitude,Longitude,City,State,Team,Conference,Capacity,Record,Built,Surface
0,Aggie Memorial Stadium,32.279486,-106.741713,Las Cruces,NM,New Mexico State,Independent,"30,343[1]","32,993(September 26, 1998 vs. UTEP)[1]",1978[1],S5-M Synthetic Turf
1,Alamodome,29.416983,-98.478814,San Antonio,TX,UTSA,C-USA,65000,"56,743(September 3, 2011 vs. Northeastern State)",1993,SportField
2,Alaska Airlines Field at Husky Stadium,47.650350,-122.301611,Seattle,WA,Washington,Pac-12,"70,083[2]","76,125(September 23, 1995 vs. Army)",1920,FieldTurf
3,Albertsons Stadium,43.602844,-116.195866,Boise,ID,Boise State,Mountain West,"36,387[3]","36,864(September 20, 2012 vs. BYU)[4]",1970[3],Smurf Turf
4,Allegiant Stadium,36.090866,-115.183322,Paradise,NV,UNLV,Mountain West,65000,NaN,2020,FieldTurf
...,...,...,...,...,...,...,...,...,...,...,...
125,Veterans Memorial Stadium at Larry Blakeney Field,31.799556,-85.951833,Troy,AL,Troy,Sun Belt,30402,"29,013(September 15, 2012 vs. Mississippi State)",1950,ProGrass
126,Waldo Stadium,42.285756,-85.601090,Kalamazoo,MI,Western Michigan,MAC,"30,200[152]","36,361(September 16, 2000 vs. Indiana State)",1939[153],FieldTurf
127,Warren McGuirk Alumni Stadium,42.377271,-72.536040,Hadley,MA,UMass,Independent,"17,000(expandable to 21,430[154])","20,000(November 25, 1972 vs. Boston College)",1965,FieldTurf
128,Wayne Day Family Field at Carter–Finley Stadium,35.800800,-78.719566,Raleigh,NC,NC State,ACC,"57,583[155]","57,583(multiple times)",1966,Tifway 419 Bermuda Grass


In [25]:
url = 'https://en.wikipedia.org/wiki/NCAA_Division_I_FBS_football_win-loss_records'
win_loss_record= pd.read_html(url)[2]
win_loss_record

,Team,Won,Lost,Tied,Pct.,Years,Total Games,Conference
0,Ohio State 1,931,327,53,0.730,130,1311,Big Ten
1,Alabama 2,929,331,43,0.729,125,1303,SEC
2,Notre Dame 3,918,328,42,0.729,130,1288,Independent
3,Boise State,465,172,2,0.729,52,639,Mountain West
4,Michigan,964,350,36,0.727,140,1350,Big Ten
...,...,...,...,...,...,...,...,...
125,UNLV 11,236,364,4,0.394,52,604,Mountain West
126,Kent State,353,570,28,0.386,97,951,MAC
127,FIU,79,143,0,0.356,18,222,C-USA
128,Charlotte,34,62,0,0.354,9,96,C-USA


In [26]:
#create sqlite database and connect to it
database_path = "Resources/college_football_db.sqlite"
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [28]:
# store dataframes into database
stadiums_cleaned_df.to_sql('stadiums', conn, if_exists='replace')
win_loss_record.to_sql('win_loss_record', conn, if_exists='replace')

In [29]:
# view tables in database
inspector = inspect(engine)
inspector.get_table_names()

['stadiums', 'win_loss_record']

In [30]:
#close database connection
conn.close()